In [1]:
import pyscf
import pyscf.tools

from orbitalpartitioning import *

In [2]:
molecule="""
S 0.04 -1.78 -1.29
S -0.04 1.78 -1.29
S 1.78 -0.04 1.29
S -1.78 0.04 1.29
Fe 0.05 -1.37 1.01
Fe -1.38 0.05 -1.00
Fe -0.05 1.38 1.00
Fe 1.37 -0.05 -1.01
S 0.24 3.30 2.14
S -0.24 -3.29 2.14
S -3.29 -0.24 -2.14
S 3.29 0.24 -2.14
C -3.80 -1.84 -1.38
H -3.91 -1.71 -0.29
H -4.76 -2.17 -1.81
H -3.03 -2.60 -1.56
C 3.80 1.83 -1.38
H 3.91 1.71 -0.29
H 4.76 2.16 -1.81
H 3.03 2.59 -1.55
C -1.83 -3.80 1.38
H -2.16 -4.76 1.81
H -2.59 -3.03 1.55
H -1.70 -3.91 0.29
C 1.84 3.80 1.38
H 2.17 4.76 1.81
H 2.60 3.03 1.56
H 1.71 3.91 0.29
"""
basis = "def2-svp"
pymol = pyscf.gto.Mole(
        atom    =   molecule,
        symmetry=   True,
        spin    =   10, # number of unpaired electrons
        charge  =   -2,
        basis   =   basis)


pymol.build()
print("symmetry: ",pymol.topgroup)
# mf = pyscf.scf.UHF(pymol).x2c()
mf = pyscf.scf.ROHF(pymol).newton()
# mf = pyscf.scf.ROHF(pymol)
mf.verbose = 4
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.chkfile = "scf.fchk"
mf.init_guess = "sad"
mf.run(max_cycle=100)

print(" Hartree-Fock Energy: %12.8f" % mf.e_tot)
# mf.analyze()
# Get data
F = mf.get_fock()
C = mf.mo_coeff
S = mf.get_ovlp()

# Just use alpha orbitals
Cdocc = mf.mo_coeff[:,mf.mo_occ==2]
Csing = mf.mo_coeff[:,mf.mo_occ==1]
Cvirt = mf.mo_coeff[:,mf.mo_occ==0]
ndocc = Cdocc.shape[1]
nsing = Csing.shape[1]
nvirt = Cvirt.shape[1]


symmetry:  C1


******** <class 'pyscf.scf.rohf.ROHF'> Newton solver flags ********
SCF tol = 1e-08
conv_tol_grad = 1e-05
max. SCF cycles = 100
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_cycle_inner = 12
max_stepsize = 0.05
ah_start_tol = 1e+09
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_start_cycle = 1
ah_max_cycle = 40
ah_grad_trust_region = 2.5
kf_interval = 4
kf_trust_region = 5
canonicalization = True
max_memory 4000 MB (current use 123 MB)
  HOMO = 0.192122760192096  LUMO = 0.206613795921749
Initial guess E= -8329.39533346297  |g|= 11.6722
macro= 0  E= -8373.45837398541  delta_E= -44.063  |g|= 8.38215  3 KF 15 JK
macro= 1  E= -8381.38156199117  delta_E= -7.92319  |g|= 1.68963  3 KF 15 JK
macro= 2  E= -8384.24246679737  delta_E= -2.8609  |g|= 0.614582  3 KF 15 JK
macro= 3  E= -8385.69433999074  delta_E= -1.45187  |g|= 0.350004  3 KF 17 JK
macro= 4  E= -8386.25797434272  delta_E= -0.563634  |g|= 0.235903  3 KF 17 JK
macro= 5  

In [5]:


nbas = Cdocc.shape[0]

In [6]:
#(36 orbital active space)
# Find AO's corresponding to atoms
full = []
frag1 = []
frag2 = []
frag3 = []
frag4 = []
frag5 = []
frag6 = []
frag7 = []
frag8 = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    if ao[0] == 0:
        if ao[2] in ("3s","3p"):
            frag1.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 1:
        if ao[2] in ("3s", "3p"):
            frag2.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 2:
        if ao[2] in ("3s", "3p"):
            frag3.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 3:
        if ao[2] in ("3s","3p"):
            frag4.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 4:
        if ao[2] in ("3d"):
            frag5.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 5:
        if ao[2] in ("3d"):
            frag6.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 6:
        if ao[2] in ("3d"):
            frag7.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 7:
        if ao[2] in ("3d"):
            frag8.append(ao_idx)
            full.append(ao_idx)
    


frags = [frag1, frag2, frag3, frag4,frag5,frag6,frag7,frag8]
print(frags)


[[2, 7, 8, 9], [20, 25, 26, 27], [38, 43, 44, 45], [56, 61, 62, 63], [86, 87, 88, 89, 90], [117, 118, 119, 120, 121], [148, 149, 150, 151, 152], [179, 180, 181, 182, 183]]


In [31]:
#(56 orbital active space)
# Find AO's corresponding to atoms
full = []
frag1 = []
frag2 = []
frag3 = []
frag4 = []
frag5 = []
frag6 = []
frag7 = []
frag8 = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    if ao[0] == 0:
        if ao[2] in ("3s","3p"):
            frag1.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 1:
        if ao[2] in ("3s", "3p"):
            frag2.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 2:
        if ao[2] in ("3s", "3p"):
            frag3.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 3:
        if ao[2] in ("3s","3p"):
            frag4.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 4:
        if ao[2] in ("3d","4d"):
            frag5.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 5:
        if ao[2] in ("3d","4d"):
            frag6.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 6:
        if ao[2] in ("3d","4d"):
            frag7.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 7:
        if ao[2] in ("3d","4d"):
            frag8.append(ao_idx)
            full.append(ao_idx)
    


frags = [frag1, frag2, frag3, frag4,frag5,frag6,frag7,frag8]
print(frags)


[[2, 7, 8, 9], [20, 25, 26, 27], [38, 43, 44, 45], [56, 61, 62, 63], [86, 87, 88, 89, 90, 91, 92, 93, 94, 95], [117, 118, 119, 120, 121, 122, 123, 124, 125, 126], [148, 149, 150, 151, 152, 153, 154, 155, 156, 157], [179, 180, 181, 182, 183, 184, 185, 186, 187, 188]]


In [33]:
# Define projectors
X = scipy.linalg.sqrtm(S)
X = np.eye(nbas) 
Pfull = X[:,full]  # non-orthogonal
Pf = []
for f in frags:
    Pf.append(X[:,f])

In [34]:
(Oact, Sact, Vact), (Cenv, Cerr, _) = svd_subspace_partitioning((Cdocc, Csing, Cvirt), Pfull, S)
assert(Cerr.shape[1] == 0)
Cact = np.hstack((Oact, Sact, Vact))

 Partition  384 orbitals into a total of   56 orbitals
            Index   Sing. Val. Space       
                0   1.35322627            2*
                1   1.34894261            2*
                2   1.34862226            2*
                3   1.33854796            2*
                4   1.33012746            2*
                5   1.32632504            2*
                6   1.31404456            2*
                7   1.30637889            2*
                8   1.30342061            2*
                9   1.29841554            2*
               10   1.29312494            2*
               11   1.27350320            2*
               12   1.26460089            2*
               13   1.25144961            2*
               14   1.20477178            2*
               15   1.18934273            2*
               16   1.18670024            2*
               17   1.18135435            2*
               18   1.16395141            2*
               19   1.13941819            2*
 

In [35]:
# Project active orbitals onto fragments
init_fspace = []
clusters = []
Cfrags = []
orb_index = 1


for fi,f in enumerate(frags):
    print()
    print(" Fragment: ", f)
    (Of, Sf, Vf), (_, _, _) = svd_subspace_partitioning((Oact, Sact, Vact), Pf[fi], S)
    Cfrags.append(np.hstack((Of, Sf, Vf)))
    ndocc_f = Of.shape[1]
    init_fspace.append((ndocc_f+Sf.shape[1], ndocc_f))
    nmof = Of.shape[1] + Sf.shape[1] + Vf.shape[1]
    clusters.append(list(range(orb_index, orb_index+nmof)))
    orb_index += nmof



# Orthogonalize Fragment orbitals
Cfrags = sym_ortho(Cfrags, S)

# Pseudo canonicalize fragments
Cfrags = canonicalize(Cfrags, F)


Cact = np.hstack(Cfrags)

# Write Molden files for visualization
pyscf.tools.molden.from_mo(mf.mol, "Pfull.molden", Pfull)
pyscf.tools.molden.from_mo(mf.mol, "Cact.molden", Cact)
pyscf.tools.molden.from_mo(mf.mol, "Cenv.molden", Cenv)
for i in range(len(frags)):
    pyscf.tools.molden.from_mo(mf.mol, "Cfrag%i.molden"%i, Cfrags[i])

print(" init_fspace = ", init_fspace)
print(" clusters    = ", clusters)




 Fragment:  [2, 7, 8, 9]
 Partition   56 orbitals into a total of    4 orbitals
            Index   Sing. Val. Space       
                0   0.97335622            0*
                1   0.82178927            0*
                2   0.81452998            0*
                3   0.79203546            0*
                4   0.38479116            2
                5   0.23367921            2
                6   0.20762315            2
                7   0.17054174            1
                8   0.13101775            1
                9   0.10338287            1
               10   0.04410571            1
               11   0.03224599            2

 Fragment:  [20, 25, 26, 27]
 Partition   56 orbitals into a total of    4 orbitals
            Index   Sing. Val. Space       
                0   0.97384930            0*
                1   0.82563769            0*
                2   0.80815328            0*
                3   0.79086085            0*
                4   0.41659542    

In [36]:
print(Cenv.shape)
print(Cact.shape)
d1_embed = 2 * Cenv @ Cenv.T

h0 = pyscf.gto.mole.energy_nuc(mf.mol)
h  = pyscf.scf.hf.get_hcore(mf.mol)
j, k = pyscf.scf.hf.get_jk(mf.mol, d1_embed, hermi=1)

print(h.shape)

(384, 108)
(384, 56)
(384, 384)


In [40]:
h0 += np.trace(d1_embed @ ( h + .5*j - .25*k))

h = Cact.T @ h @ Cact;
j = Cact.T @ j @ Cact;
k = Cact.T @ k @ Cact;
print(h.shape)

(56, 56)


In [41]:
nact = h.shape[0]
print(nact)
h2 = pyscf.ao2mo.kernel(pymol, Cact, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)

56


In [ ]:
# The use of d1_embed only really makes sense if it has zero electrons in the
# active space. Let's warn the user if that's not true

S = pymol.intor("int1e_ovlp_sph")
n_act = np.trace(S @ d1_embed @ S @ Cact @ Cact.T)
if abs(n_act) > 1e-8 == False:
    print(n_act)
    error(" I found embedded electrons in the active space?!")

h1 = h + j - .5*k;

In [ ]:
np.save("ints_h0_fe4s4_56", h0)
np.save("ints_h1_fe4s4_56", h1)
np.save("ints_h2_fe4s4_56", h2)
np.save("mo_coeffs_fe4s4_56", Cact)
np.save("overlap_mat_fe4s4_56", S)

Pa = mf.make_rdm1()[0]
Pb = mf.make_rdm1()[1]
np.save("Pa_fe4s4_56", Cact.T @ S @ Pa @ S @ Cact)
np.save("Pb_fe4s4_56", Cact.T @ S @ Pb @ S @ Cact)

In [16]:
import numpy as np
Ccmf = np.load("Ccmf_fe4s4_56.npy")
pyscf.tools.molden.from_mo(mf.mol, "Ccmf_fe4s4_56.molden", Ccmf)